In [4]:
import requests
import re
from bs4 import BeautifulSoup
from urllib import request
from urllib.request import Request, urlopen
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

In [64]:
#scrap the category links from the navigation bar/pane
def scrap_navlinks(url):
  page_request = request.Request(url, headers=agent)
  page = request.urlopen(page_request)
  soup = BeautifulSoup(page, 'html.parser')
  #The daily sun news website has a navigation bar with the CSS class attribute containing the texts below
  xp = soup.find("li", {"class": re.compile("^menu-item menu-item-type-taxonomy menu-item-object-category menu-item")}).findAll('a')
  navlinks = list(set([i['href'] for i in xp if 'https://dailytimes.ng/category/' in i['href']]))

  return navlinks

#scrap links from the different categories 
# if it it return any status code aside 200 for 5 time consecutively, it retuns whatever links are available
def scrap_link(catgory, catgory_size=10000):
  urls = []
  print("Getting the URL from the category : ", catgory, " (featured) with ", catgory_size, " pages . " )
  error = 0 #keep track of the error
  for p in range(catgory_size):
    url = catgory+'page/'+str(p+1)+'/'
    print(url)
    reqs = requests.get(url, headers=agent)
    soup = BeautifulSoup(reqs.content, 'lxml')
    if reqs.status_code != 200:
      error +=1
    if error >= 5:
      break
    res = soup.find_all('h2',{'class':'entry-title'})#.find_all("a", href=True)
    #print(res)
    urls.extend([t.a['href'] for t in res])
  #return all the news article links from the category
  return urls

In [68]:
#add the URL here 
url = 'https://dailytimes.ng'
#scrap the category links from the navigation pane
navlinks = scrap_navlinks(url)
print('Got ',len(navlinks), ' categories from the navigation bar.')
print(navlinks)
urls = [scrap_link(navlinks[i]) for i in range(len(navlinks))] #returns a list of lists
#flatten the list of lists, that is urls
urls = set([y for x in urls for y in x])

Got  13  categories from the navigation bar.
['https://dailytimes.ng/category/history/', 'https://dailytimes.ng/category/lifestyle/', 'https://dailytimes.ng/category/editorial/', 'https://dailytimes.ng/category/news/', 'https://dailytimes.ng/category/metro/', 'https://dailytimes.ng/category/sport/', 'https://dailytimes.ng/category/tech/', 'https://dailytimes.ng/category/health/', 'https://dailytimes.ng/category/opinion/', 'https://dailytimes.ng/category/business/', 'https://dailytimes.ng/category/entertainment/', 'https://dailytimes.ng/category/extras/', 'https://dailytimes.ng/category/politics/']
Getting the URL from the category :  https://dailytimes.ng/category/history/  (featured) with  10000  pages . 
https://dailytimes.ng/category/history/page/1/
Getting the URL from the category :  https://dailytimes.ng/category/lifestyle/  (featured) with  10000  pages . 
https://dailytimes.ng/category/lifestyle/page/1/


In [71]:
#scrap the articles with links in the `urls` variable
def getSoup(links):
  print("getting soup objects for all the links ... ")
  soupx = []
  cnt=0
  for url in links: 
    page_request = request.Request(url, headers=agent)
    page = request.urlopen(page_request)
    #response = requests.get(page_url)
    #print(page.getcode())
    soup = BeautifulSoup(page, 'html.parser')
    soupx.append(soup)
    cnt = cnt + 1
    progresse = (cnt/len(links) * 100) 
    if progresse  % 10 == 0:
      print ("Got ", progresse, "% of soup objects")
  print("got the soup object for all the links ... ")
  return soupx
#get the news content from the soup objects extracted above (return the title, date and texts)
def getcontent(soupx):
  print("scrapping the articles ... ")
  cnt=0;
  title=[];time=[]; texts=[]
  for soups in soupx:
    cnt=cnt+1
    if soups.find("h1", {"class":"entry-title"})!= None:
      title.append(soups.find("h1", {"class":"entry-title"}).text.replace("\t","").replace("\n",""))
    else:
      title.append("")
    if soups.find("span", {"class":"updated"})!= None:
      time.append(soups.find("span", {"class":"updated"}).text.replace("\t","").replace("\n",""))
    else:
      time.append("")
  
    if soups.find("div", {"class":"entry-content"})!=None:
      result = soups.find("div", {"class":"entry-content"}).findAll('p')
      txtstring=""
      for x in result:
      #print (x.text)
        txtstring+=x.text.replace(u'\xa0', u' ').replace('\n'," ")+" \n"
      texts.append(txtstring.strip())
    else:
      texts.append("") 

    progresse = (cnt/len(soupx) * 100) 
    if progresse  % 10 == 0:
      print ("Scrapped ", progresse, "% of the articles")

  return title,time,texts

In [72]:
soups = getSoup(list(urls))
title,time,texts = getcontent(soups)
#create a dictionarty
import pandas as pd
d = {'Date':time,'Title':title,'Text':texts}
df = pd.DataFrame(d)

getting soup objects for all the links ... 
Got  50.0 % of soup objects
Got  100.0 % of soup objects
got the soup object for all the links ... 
scrapping the articles ... 
Scrapped  50.0 % of the articles
Scrapped  100.0 % of the articles


In [74]:
print(df)

,Date,Title,Text
0,15th April 2021,Boost Your Confidence,"By Mutiat Alli \n1. To feel, act and appear co..."
1,30th October 2020,Burna Boy releases new single to honour victim...,International multiple award-winning superstar...
2,11th September 2020,"History: The story of Ezego, Nigeria’s King of...","Today, history remembers the story of Victor O..."
3,3rd December 2020,6 killed in building collapse in northern Egypt,At least six persons were killed in a building...
4,16th October 2020,"Bangladesh High Commission lauds role of arts,...",By Ebere Chibuzor \nThe outgoing High Commissi...
5,27th April 2021,EU may impose sanctions if London fails to hon...,The European Union (EU) could impose retaliato...
6,27th April 2021,Britain’s journalists hit back at PM over ‘bod...,Britain’s journalists and media have hit back ...
7,6th October 2020,Benefits of drinking hot water,By Mutiat Alli \nMost of us know by now that w...
8,26th September 2020,Simple morals you develop at home,By Folarin Leonard \nThe saying word “Charity”...
9,9th October 2020,"COVID-19: Sanwo-olu grants Funke Akindele, hus...","The Governor of Lagos State, Mr Babajide Sanwo..."


In [ ]:
#write the dataframe to file